In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
from d2lvit import *
import copy
from collections import OrderedDict

In [2]:
unused_digit_model = nn.Linear(1,1)

In [3]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [4]:
model = models.vit_b_16(weights="ViT_B_16_Weights.IMAGENET1K_V1")

In [5]:
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 45))]))
model = model.to(device)

In [6]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[-0.1356,  0.3515,  0.2299,  0.0983, -0.2027,  0.1818, -0.1348, -0.0159,
         -0.3947, -0.4792, -0.0490,  0.3720,  0.0714, -0.2446,  0.2968,  0.5886,
         -0.5627, -0.2200, -0.1116, -0.7799,  0.7426, -0.0468, -0.0398, -0.5055,
          0.0103, -0.2745, -0.2472, -0.4506,  0.4544,  0.2546,  0.1421, -0.1624,
          0.1161,  0.0595,  0.9386, -0.0024, -0.2661, -0.2394,  0.3706,  0.5595,
          0.1716, -0.5610,  0.0358, -0.1817, -0.2941]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [7]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'finetune_scratchtrees'), unused_digit_model, False, False)

Epoch 10 done, train loss: 0.0009 val acc: 1.0000
Epoch 20 done, train loss: 0.0002 val acc: 1.0000
Epoch 30 done, train loss: 0.0001 val acc: 1.0000
Epoch 40 done, train loss: 0.0000 val acc: 1.0000
Epoch 50 done, train loss: 0.0000 val acc: 1.0000
Epoch 60 done, train loss: 0.0000 val acc: 1.0000
Epoch 70 done, train loss: 0.0000 val acc: 1.0000
Epoch 80 done, train loss: 0.0000 val acc: 1.0000
Epoch 90 done, train loss: 0.0000 val acc: 1.0000
Epoch 100 done, train loss: 0.0000 val acc: 1.0000


In [8]:
model = torch.load(os.path.join('..', 'models', 'finetune_scratchtrees', 'tree-model.pt'))

In [9]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, unused_digit_model, False, False)
print(train_acc)

1.0


In [10]:
val_acc = m.predict(model, val_loader, device, config, unused_digit_model, False, False)
print(val_acc)

1.0
